In [ ]:
import pypsa
import yaml
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import cartopy.crs as ccrs
import cartopy

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from matplotlib.patches import Circle, Patch
from matplotlib.legend_handler import HandlerPatch

from pypsa.descriptors import get_switchable_as_dense as as_dense

from shapely import wkt
import sys, os

PATH = "../../../../"
SCRIPTS_PATH = "pypsa-eur-sec/scripts/"

sys.path.append(os.path.join(PATH, SCRIPTS_PATH))
# from plot_network import make_legend_circles_for, assign_location
# from helper import override_component_attrs
# from build_gas_input_locations import build_gas_input_locations, load_bus_regions

#plt.style.use(["bmh", "matplotlibrc"])
xr.set_options(display_style="html")

%matplotlib inline

In [ ]:
OUTPUT='/mnt/c/Users/scl38887/Documents/git/aldehyde/results/graphics/'

#fn = f"{PATH}resources/bus_regions/regions_onshore_elec_s_4.geojson"
fn = '/mnt/c/Users/scl38887/Documents/git/pypsa-earth/resources/bus_regions/regions_onshore_elec_s_4.geojson'
nodes = gpd.read_file(fn).set_index("name")

# Import the network representing MAR ES only
n = pypsa.Network('../../resources/mar_es_only/elec_s_4_ec_lc1.0_Co2L_144H_2030_0.071_DF_nosmr.nc')
print("The network representing MAR ES only 'n1' has been imported")

### Data preparation

In [ ]:
# Insert hack to copy index of all rows of n.buses with carrier "AC" to the column "location" and avoid length mismatch. They were missing
n.buses.loc[n.buses.carrier == "AC", "location"] = n.buses.index[n.buses.carrier == "AC"]

### Capacity Factors Renewables

In [ ]:
df = (
    n.generators_t.p_max_pu.mean()
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .mul(100)
)

In [ ]:
techs = [
    #"offwind-ac",
    #"offwind-dc",
    "onwind",
    "solar",
    #"solar thermal",
    #"ror",
    #"air-sourced heat pump",
    #"ground-sourced heat pump",
    "residential rural solar thermal",
]
df = df[techs]

In [ ]:
def plot_capacity_factors(
    df,
    geodf,
    carrier,
    cmap="Blues",
    vmax=100,
    vmin=0,
    label="capacity factors [%]",
    fn=None,
):

    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmax=vmax,
        vmin=vmin,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            # "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    #plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor("white")

    if fn is None:
        plt.savefig(f"{OUTPUT}/cf-{carrier}.pdf", bbox_inches="tight")

In [ ]:
plot_capacity_factors(df, nodes, "onwind", vmax=55)

In [ ]:
plot_capacity_factors(df, nodes, "solar", "Oranges", vmax=max(df["solar"]))

In [ ]:
plot_capacity_factors(df, nodes, "residential rural solar thermal", "Oranges", vmax=15)

### Potentials Renewables

In [ ]:
df = (
    n.generators.p_nom_max
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .div(1e3) # in GW
)

In [ ]:
def plot_potentials(
    df,
    geodf,
    carrier,
    cmap="Blues",
    vmax=100,
    vmin=0,
    label="Potentials in GW",
    fn=None,
):

    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmax=vmax,
        vmin=vmin,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            # "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    #plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor("white")

    if fn is None:
        plt.savefig(f"{OUTPUT}/p-nom-max-{carrier}.pdf", bbox_inches="tight")

In [ ]:
plot_potentials(df, nodes, "onwind", vmax=max(df["onwind"]))

In [ ]:
plot_potentials(df, nodes, "solar", "Oranges", vmax=max(df["solar"]))

### Optimised Cap Renewables

In [ ]:
df = (
    n.generators.p_nom_opt
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .div(1e3) # in GW
)

In [ ]:
def plot_opt_cap(
    df,
    geodf,
    carrier,
    cmap="Blues",
    vmax=100,
    vmin=0,
    label="Optimised capacity in GW",
    fn=None,
):

    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmax=vmax,
        vmin=vmin,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            # "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    #plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor("white")

    if fn is None:
        plt.savefig(f"{OUTPUT}/p-nom-opt-{carrier}.pdf", bbox_inches="tight")

In [ ]:
plot_opt_cap(df, nodes, "onwind", vmax=max(df["onwind"]))

In [ ]:
plot_opt_cap(df, nodes, "solar", "Oranges", vmax=max(df["solar"]))

### Usage Renewables

In [ ]:
df = (
    (n.generators.p_nom_opt/n.generators.p_nom_max)
    .groupby([n.generators.carrier, n.generators.bus.map(n.buses.location)])
    .first()
    .unstack(0)
    .mul(1e2) # in %
)

In [ ]:
def plot_usage(
    df,
    geodf,
    carrier,
    cmap="Blues",
    vmax=100,
    vmin=0,
    label="Usage in %",
    fn=None,
):

    proj = ccrs.EqualEarth()
    geodf = geodf.to_crs(proj.proj4_init)

    fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": proj})

    geodf.plot(
        ax=ax,
        column=df[carrier].reindex(geodf.index),
        cmap=cmap,
        linewidths=0,
        legend=True,
        vmax=vmax,
        vmin=vmin,
        legend_kwds={
            "label": label,
            "shrink": 0.7,
            # "extend": "max",
        },
    )

    ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
    ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)

    #plt.gca().outline_patch.set_visible(False)
    ax.set_facecolor("white")

    if fn is None:
        plt.savefig(f"{OUTPUT}/usage-{carrier}.pdf", bbox_inches="tight")

In [ ]:
plot_usage(df, nodes, "onwind", vmax=max(df["onwind"]))

In [ ]:
plot_usage(df, nodes, "solar", "Oranges", vmax=max(df["solar"]))